# NLP - Text preprocessing
The pipeline steps were based on the following sources:
- [https://sol.sbc.org.br/index.php/stil/article/download/31163/30966/](https://sol.sbc.org.br/index.php/stil/article/download/31163/30966/)

In [1]:
import pandas as pd
import csv

# Import the corpus from a CSV
df = pd.read_csv('data/DepreRedditBR.csv')

In [3]:
df.head()
df.columns

Index(['text'], dtype='object')

In [4]:
df = df.sample(frac=0.1, random_state=42)

df

,text
208601,não há nada que eu queira mais do que voltar n...
462528,Lidando com mudanças de humor Tenho tido muita...
76680,"Se sua noiva não usar com voce, ja sabe. Não p..."
359032,muitas vezes não consigo trabalhar cerca de um...
353889,quais são suas ações quando você está se senti...
...,...
271068,só estou vivo para outras pessoas e estou faze...
55861,Parabéns amigo! Você é o tapa buraco. Agora de...
339906,"ansiedade por estar doente, sou epiléptico e m..."
178869,Passei por um período muito difícil durante ...


In [5]:
# Replace comma with 'chavevirg' label
df = df.apply(lambda text: text.str.replace(r',', 'chavevirg'), axis=1)
df

,text
208601,não há nada que eu queira mais do que voltar n...
462528,Lidando com mudanças de humor Tenho tido muita...
76680,Se sua noiva não usar com vocechavevirg ja sab...
359032,muitas vezes não consigo trabalhar cerca de um...
353889,quais são suas ações quando você está se senti...
...,...
271068,só estou vivo para outras pessoas e estou faze...
55861,Parabéns amigo! Você é o tapa buraco. Agora de...
339906,ansiedade por estar doentechavevirg sou epilép...
178869,Passei por um período muito difícil durante ...


In [8]:
# Enelvo Text normaliser (https://thalesbertaglia.com/enelvo/)
import re
from enelvo.normaliser import Normaliser

normaliser_instance = Normaliser(tokenizer='readable')
df = df.apply(lambda row: normaliser_instance.normalise(row.text), axis=1)

# Remove URLs if there is any
df = df.apply(lambda text: re.sub(r'http\S+', '', text))

df

KeyboardInterrupt: 